In [1]:
from langchain_community.llms import Ollama
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
llm = Ollama(model='llama2')
llm.invoke("the first man on moon was?")

'\nThe first man to walk on the Moon was Neil Armstrong. He stepped out of the Apollo 11 lunar module Eagle and onto the Moon\'s surface on July 20, 1969, famously declaring "That\'s one small step for man, one giant leap for mankind" as he took his first steps. Armstrong was accompanied by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the Moon during the mission.'

## Read Transaction Data

In [3]:
df = pd.read_csv('data/transactions_2022_2023.csv')
df

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08
7,2023-12-22,Consulting,Income,541.57
8,2023-12-22,Aidsfonds,Expense,10.70
9,2023-12-20,Consulting,Income,2641.93


In [4]:
unique_transactions = df['Name / Description'].unique()
len(unique_transactions)

23

In [5]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

## Categorise Bank Transactions

In [7]:
def hop(start, stop, step):
    for i in range (start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [8]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [9]:
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Of course! Here are the appropriate categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food & Beverage', '4. Taxi Utrecht - Transportation', '5. Etos AMSTERDAM NLD - Grocery', '6. Bistro Bar Amsterdam - Food & Beverage']


,Transaction vs category,Transaction,Category
0,,,None
1,Of course! Here are the appropriate categories...,Of course! Here are the appropriate categories...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
5,3. ISS Catering Services De Meern - Food & Bev...,3. ISS Catering Services De Meern,Food & Beverage
6,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
7,5. Etos AMSTERDAM NLD - Grocery,5. Etos AMSTERDAM NLD,Grocery
8,6. Bistro Bar Amsterdam - Food & Beverage,6. Bistro Bar Amsterdam,Food & Beverage


In [10]:
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Sure! Here are the appropriate categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food & Beverage', '4. Taxi Utrecht - Transportation', '5. Etos AMSTERDAM NLD - Groceries', '6. Bistro Bar Amsterdam - Dining']


,Transaction vs category,Transaction,Category
0,,,None
1,Sure! Here are the appropriate categories for ...,Sure! Here are the appropriate categories for ...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
5,3. ISS Catering Services De Meern - Food & Bev...,3. ISS Catering Services De Meern,Food & Beverage
6,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
7,5. Etos AMSTERDAM NLD - Groceries,5. Etos AMSTERDAM NLD,Groceries
8,6. Bistro Bar Amsterdam - Dining,6. Bistro Bar Amsterdam,Dining


In [11]:
categories_df_all = pd.DataFrame()

for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Certainly! Here are the appropriate categories for each of the expenses you provided:', '', '1. Belastingdienst - Taxes', '2. Tesco Breda - Groceries', '3. Monthly Appartment Rent - Housing', '4. Vishandel Sier Amsterdam - Shopping', '5. Selling Paintings - Art Supplies', '6. Spotify Ab By Adyen - Entertainment', '7. Tk Maxx Amsterdam Da - Clothing', '8. Consulting - Professional Services', '9. Aidsfonds - Charity Donations', '10. Tls Bv Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Health and Beauty', '12. Beta Boulders Ams Amsterdam - Fitness', '13. Salary - Personal Expenses', '14. Bouldermuur Bv Amsterdam - Home Improvements', '15. Birtat Restaurant Amsterdam - Dining Out', '16. Freelancing - Business Expenses', '17. Tikkie - Financial Services', '18. Blogging - Personal Interest', '19. Taxi Utrecht - Transportation', '20. Apple Services - Tech and Gadgets', '21. Amazon Lux - Online Shopping', '22. Classpass* Monthly - Fitness Subscription', '23. Audible Uk AdblCo/Pym

In [12]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Certainly! Here are the appropriate categories...,Certainly! Here are the appropriate categories...,None
1,,,None
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Shopping,4. Vishandel Sier Amsterdam,Shopping
6,5. Selling Paintings - Art Supplies,5. Selling Paintings,Art Supplies
7,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Clothing,7. Tk Maxx Amsterdam Da,Clothing
9,8. Consulting - Professional Services,8. Consulting,Professional Services


In [13]:
unique_categories = categories_df_all['Category'].unique()
unique_categories

array([None, 'Taxes', 'Groceries', 'Housing', 'Shopping', 'Art Supplies',
       'Entertainment', 'Clothing', 'Professional Services',
       'Charity Donations', 'Transportation', 'Health and Beauty',
       'Fitness', 'Personal Expenses', 'Home Improvements', 'Dining Out',
       'Business Expenses', 'Financial Services', 'Personal Interest',
       'Tech and Gadgets', 'Online Shopping', 'Fitness Subscription',
       'Audiobooks'], dtype=object)

In [14]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [15]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

,Transaction vs category,Transaction,Category
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
4,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
5,4. Vishandel Sier Amsterdam - Shopping,4. Vishandel Sier Amsterdam,Shopping
6,5. Selling Paintings - Art Supplies,5. Selling Paintings,Art Supplies
7,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Clothing,7. Tk Maxx Amsterdam Da,Clothing
9,8. Consulting - Professional Services,8. Consulting,Services
10,9. Aidsfonds - Charity Donations,9. Aidsfonds,Charity Donations
11,10. Tls Bv Inz Ov-Chipkaart - Transportation,10. Tls Bv Inz Ov-Chipkaart,Transportation


In [16]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("data/transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [17]:
df.to_csv("output_transactions_2022_2023_categorized.csv", index=False)